In [1]:
import pandas as pd

!pip install requests
!pip install tabulate

!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o


In [2]:
import h2o

In [11]:
from h2o.automl import H2OAutoML

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.22" 2024-01-16; OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1); OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1, mixed mode, sharing)
  Starting server from /home/mardoniofranca/work/env/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpvi8_si69
  JVM stdout: /tmp/tmpvi8_si69/h2o_mardoniofranca_started_from_python.out
  JVM stderr: /tmp/tmpvi8_si69/h2o_mardoniofranca_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,1 month and 14 days
H2O_cluster_name:,H2O_from_python_mardoniofranca_noltt4
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.840 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [7]:
h2o.upload_file("data/dados_sc_1.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


c_Order_Priority,h_Order_Priority,l_Order_Priority,m_Order_Priority,n_Order_Priority,Order_Quantity,Sales,Target
0,1,0,0,0,0.408163,0.100278,0
0,1,0,0,0,0.693878,0.122267,1
0,0,0,0,1,0.122449,0.0734383,0
0,0,1,0,0,0.102041,0.0324473,0
0,0,0,1,0,0.469388,0.0419286,0
0,0,0,0,1,0.897959,0.00826893,0
1,0,0,0,0,0.326531,0.0491602,1
0,0,1,0,0,0.408163,0.159864,1
0,0,0,0,1,0.795918,0.690681,0
0,0,0,0,1,0.326531,0.0891865,1


In [8]:
# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("data/dados_1_train.csv")
test = h2o.import_file("data/dados_1_test.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [9]:
x = train.columns
y = "Target"
x.remove(y)

In [13]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models
aml = H2OAutoML(max_models=40, seed=1)
aml.train(x=x, y=y, training_frame=train)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_1_AutoML_2_20240427_160333,0.78151,0.565524,0.761712,0.293397,0.4364,0.190445
XGBoost_grid_1_AutoML_2_20240427_160333_model_7,0.780477,0.564658,0.763662,0.281304,0.436355,0.190406
StackedEnsemble_AllModels_1_AutoML_2_20240427_160333,0.779363,0.566285,0.761414,0.298757,0.437218,0.19116
GBM_grid_1_AutoML_2_20240427_160333_model_5,0.776419,0.570129,0.742954,0.299015,0.438914,0.192645
XGBoost_grid_1_AutoML_2_20240427_160333_model_2,0.774085,0.571794,0.730518,0.304187,0.439674,0.193314
GBM_2_AutoML_2_20240427_160333,0.771077,0.574516,0.748199,0.317933,0.441587,0.194999
GBM_3_AutoML_2_20240427_160333,0.769306,0.576081,0.740881,0.300575,0.442564,0.195863
XGBoost_grid_1_AutoML_2_20240427_160333_model_1,0.767757,0.57978,0.724085,0.309289,0.443021,0.196267
GBM_grid_1_AutoML_2_20240427_160333_model_2,0.767294,0.574819,0.739559,0.30034,0.441218,0.194674
GBM_4_AutoML_2_20240427_160333,0.76689,0.579554,0.73236,0.293373,0.443839,0.196993


https://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/intro.html

https://medium.com/h2o-ai-brasil/h2o-os-primeiros-passos-fae39077e028

https://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html

https://docs.h2o.ai/h2o/latest-stable/h2o-docs/training-models.html